# Grupo 1 - Smog predicition
## Random Forest

### Análisis y limpieza de datos

In [1]:
#General imports
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn import preprocessing
from sklearn.pipeline import Pipeline

#Specific imports
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import classification_report, recall_score, precision_score, make_scorer
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from scipy.stats import sem

#Visualization
sns.set(color_codes=True)

%matplotlib inline

df = pd.read_csv('data/train.csv')

df['Gears'] = df['Transmission'].str.extract('(\d+)')
df['Gears'] = pd.to_numeric(df['Gears'], errors='coerce')
df['Transmission'] = df['Transmission'].str.extract('(\D+)')

In [2]:
#Fuel Type
df.loc[df["Fuel Type"] == "X", "Fuel Type"] = 0
df.loc[df["Fuel Type"] == "Z", "Fuel Type"] = 1
df.loc[df["Fuel Type"] == "D", "Fuel Type"] = 2
df.loc[df["Fuel Type"] == "E", "Fuel Type"] = 3
df.loc[df["Fuel Type"] == "N", "Fuel Type"] = 4

#Transmission
df.loc[df["Transmission"] == "A", "Transmission"] = 0
df.loc[df["Transmission"] == "AM", "Transmission"] = 1
df.loc[df["Transmission"] == "AS", "Transmission"] = 2
df.loc[df["Transmission"] == "AV", "Transmission"] = 3
df.loc[df["Transmission"] == "M", "Transmission"] = 4


#Vehicle Class
df.loc[df["Vehicle Class"] == "Compact", "Vehicle Class"] = 0
df.loc[df["Vehicle Class"] == "Full-size", "Vehicle Class"] = 1
df.loc[df["Vehicle Class"] == "Mid-size", "Vehicle Class"] = 2
df.loc[df["Vehicle Class"] == "Minicompact", "Vehicle Class"] = 3
df.loc[df["Vehicle Class"] == "Minivan", "Vehicle Class"] = 4
df.loc[df["Vehicle Class"] == "Minicompact", "Vehicle Class"] = 5
df.loc[df["Vehicle Class"] == "Pickup truck: Small", "Vehicle Class"] = 6
df.loc[df["Vehicle Class"] == "Pickup truck: Standard", "Vehicle Class"] = 7
df.loc[df["Vehicle Class"] == "SUV: Small", "Vehicle Class"] = 8
df.loc[df["Vehicle Class"] == "SUV: Standard", "Vehicle Class"] = 9
df.loc[df["Vehicle Class"] == "Special purpose vehicle", "Vehicle Class"] = 10
df.loc[df["Vehicle Class"] == "Station wagon: Mid-size", "Vehicle Class"] = 11
df.loc[df["Vehicle Class"] == "Station wagon: Small", "Vehicle Class"] = 12
df.loc[df["Vehicle Class"] == "Subcompact", "Vehicle Class"] = 13
df.loc[df["Vehicle Class"] == "Two-seater", "Vehicle Class"] = 14

df = df.dropna(subset=['Gears'])

In [3]:
df.drop("Model Year", axis=1, inplace=True)
df.drop("Make", axis=1, inplace=True)
df.drop("Model", axis=1, inplace=True)
df.drop("Comb (mpg)", axis=1, inplace=True)
df.drop("Fuel Consumption City (L/100 km)", axis=1, inplace=True)
df.drop("Hwy (L/100 km)", axis=1, inplace=True)

df.head()

,id,Vehicle Class,Engine Size (L),Cylinders,Transmission,Fuel Type,Comb (L/100 km),CO2 Emissions (g/km),Smog,Gears
0,ab44e9bec15,12,2.0,4,1,1,8.7,202,2,7.0
1,45926762371,2,2.0,4,2,0,7.7,181,4,6.0
2,e9be56e153f,1,2.9,6,1,1,11.7,274,2,8.0
3,077092760df,0,2.0,4,2,0,8.1,189,1,6.0
4,c1c2579b795,3,5.2,12,0,1,13.8,324,1,8.0


Con esto, hemos terminado el análisis y la limpieza de los datos del dataframe de entrenamiento.

### Definición y entrenamiento del modelo

In [4]:
features = ['Vehicle Class', 'Engine Size (L)', 'Cylinders', 'Transmission', 'Fuel Type', 'CO2 Emissions (g/km)', 'Gears']

x = df[features].values
y = df['Smog'].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=33)

# Preprocess: normalize
scaler = preprocessing.StandardScaler().fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [5]:
model = RandomForestClassifier(n_estimators=70, random_state=100)

model.fit(x_train, y_train)

RandomForestClassifier(n_estimators=70, random_state=100)

### Comprobación de resultados

Ahora, se van a observar los resultados del modelo entrenado.

In [6]:
# Evaluate Accuracy in training
predicted = model.predict(x_test)
expected = y_test

from sklearn import metrics
y_train_pred = model.predict(x_train)
print("Accuracy in training", metrics.accuracy_score(y_train, y_train_pred))

# Now we evaluate error in testing
y_test_pred = model.predict(x_test)
print("Accuracy in testing ", metrics.accuracy_score(y_test, y_test_pred))

Accuracy in training 0.9905437352245863
Accuracy in testing  0.7872340425531915


Obtenemos una exactitud del 78%. Mucho mejor que en los casos anteriores.

In [7]:
s_y_test = Series(y_test)
s_y_test.value_counts()

s_y_test.value_counts().head(1) / len(y_test)

2    0.35461
dtype: float64

Además, el modelo no se encuentra sobreajustado.

In [8]:
print(classification_report(expected, predicted))

              precision    recall  f1-score   support

           0       0.86      1.00      0.92         6
           1       0.77      0.74      0.75        27
           2       0.78      0.78      0.78        50
           3       0.87      0.83      0.85        24
           4       0.74      0.76      0.75        34

    accuracy                           0.79       141
   macro avg       0.80      0.82      0.81       141
weighted avg       0.79      0.79      0.79       141



Estas métricas también son bastante mejores que en el resto de modelos, obteniendo una F1-Score aceptable.

Ahora, vamos a probar a entrenarlo y evaluarlo con K-Fold para ver si puede mejorar los resultados obtenidos.

In [9]:

cv = KFold(n_splits=5, shuffle=True, random_state=33)

scores = cross_val_score(model, x, y, cv=cv)
print("Scores in every iteration", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Scores in every iteration [0.76106195 0.80530973 0.73451327 0.73451327 0.77678571]
Accuracy: 0.76 (+/- 0.05)


Utilizando KFold se obtiene una exactitud del 76%. 

### Ajuste del algoritmo

Se va a utilizar Grid Search para realizar una optimización de hiperparámetros.

In [10]:
# set of hyperparameters to test
tuned_hyperparameters = [{'n_estimators': [50, 100, 200],
                          'max_depth': [None, 10, 20],
                          'min_samples_split': [2, 5, 10],
                          'min_samples_leaf': [1, 2, 4],
                          'max_features': ['auto', 'sqrt', 'log2'],
                          'class_weight': ['balanced', None]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyperparameters for %s" % score)
    print()

    if score == 'precision':
        scorer = make_scorer(precision_score, average='weighted', zero_division=0)
    elif score == 'recall':
        scorer = make_scorer(recall_score, average='weighted', zero_division=0)

    gs = GridSearchCV(RandomForestClassifier(), tuned_hyperparameters, cv=10, scoring=scorer)
    gs.fit(x_train, y_train)

    print("Best hyperparameters set found on development set:")
    print()
    print(gs.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = gs.cv_results_['mean_test_score']
    stds = gs.cv_results_['std_test_score']

    for mean_score, std_score, params in zip(means, stds, gs.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean_score, std_score * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, gs.predict(x_test)
    print(classification_report(y_true, y_pred))
    print()


# Tuning hyperparameters for precision

Best hyperparameters set found on development set:

{'class_weight': 'balanced', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

Grid scores on development set:

0.787 (+/-0.116) for {'class_weight': 'balanced', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
0.786 (+/-0.109) for {'class_weight': 'balanced', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
0.798 (+/-0.100) for {'class_weight': 'balanced', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
0.759 (+/-0.101) for {'class_weight': 'balanced', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
0.769 (+/-0.111) for {'class_weight': 'balanced', 'max_depth': None, 'max_features': 'auto',

Hay que destacar que para realizar el ajuste de parámetros se han necesitado 20 minutos de ejecución.

In [11]:
# We print the score for each value of max_depth
for i, max_depth in enumerate(gs.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (gs.cv_results_['mean_test_score'][i],
                                        gs.cv_results_['std_test_score'][i] * 2,
                                        max_depth))

0.770 (+/-0.081) for {'class_weight': 'balanced', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
0.766 (+/-0.118) for {'class_weight': 'balanced', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
0.761 (+/-0.090) for {'class_weight': 'balanced', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
0.728 (+/-0.137) for {'class_weight': 'balanced', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 50}
0.754 (+/-0.120) for {'class_weight': 'balanced', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
0.749 (+/-0.087) for {'class_weight': 'balanced', 'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
0.716 (+/-0.084) for {'class_weight'

In [12]:
# Define tu pipeline con RandomForest utilizando los mejores parámetros encontrados
model = Pipeline([
    ('scaler', StandardScaler()),
    ('random_forest', RandomForestClassifier(**gs.best_params_, random_state=100))
])

# Ajusta el modelo
model.fit(x_train, y_train)

# Crea un iterador de validación cruzada con k = 10 folds estratificados
cv = KFold(10, shuffle=True, random_state=33)

# Puntuaciones de validación cruzada
scores = cross_val_score(model, x, y, cv=cv)
def mean_score(scores):
    return ("Mean score: {0:.3f} (+/- {1:.3f})").format(np.mean(scores), sem(scores))
print(mean_score(scores))


Mean score: 0.789 (+/- 0.015)


El resultado es ligeramente mejor que el demostrado antes del ajuste del algoritmo. En cambio no hay casi diferencia con el mismo.